In [6]:
import numpy as np
import pandas as pd

In [7]:
train = pd.read_csv("/kaggle/input/amex-bowl-hackathon/663e2b6d54457_train_data_with_samplefeatures.csv")

bowler= pd.read_csv("/kaggle/input/amex-bowl-hackathon/663e2b2c60743_bowler_level_scorecard.csv")
batter=pd.read_csv("/kaggle/input/amex-bowl-hackathon/663e2b548c98c_batsman_level_scorecard.csv")
match =pd.read_csv("/kaggle/input/amex-bowl-hackathon/664389efa0868_match_level_scorecard.csv")
test = pd.read_csv("/kaggle/input/amex-bowl-hackathon/6644a1e287df6_test_data_with_samplefeatures.csv")

In [8]:
print(bowler.columns)
print(test.columns)

Index(['match id', 'bowler', 'bowler_id', 'bowler_details',
       'is_bowler_captain', 'is_bowler_keeper', 'inning', 'runs',
       'wicket_count', 'balls_bowled', 'economy', 'maiden', 'dots', 'Fours',
       'Sixes', 'wides', 'noballs', 'match_dt'],
      dtype='object')
Index(['match id', 'team1', 'team1_id', 'team1_roster_ids', 'team2',
       'team2_id', 'team2_roster_ids', 'toss winner', 'toss decision', 'venue',
       'city', 'match_dt', 'lighting', 'series_name', 'season', 'ground_id',
       'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15'],
      dtype='object')


In [9]:
# # Convert 'match_dt' column to datetime format
# train['match_dt'] = pd.to_datetime(train['match_dt'])
# bowler['match_dt'] = pd.to_datetime(bowler['match_dt'])
# batter['match_dt'] = pd.to_datetime(batter['match_dt'])
# match['match_dt'] = pd.to_datetime(match['match_dt'])
# test['match_dt']=pd.to_datetime(test['match_dt'])

# # Sort the DataFrame by 'match_dt'
# train = train.sort_values(by='match_dt')
# bowler = bowler.sort_values(by='match_dt')
# batter = batter.sort_values(by='match_dt')
# match = match.sort_values(by='match_dt')
# test = test.sort_values(by='match_dt')

# Train preprocess

In [10]:
for column in train.columns:
        if train[column].isnull().any():  
            train[column] = train[column].fillna(
                train[column].rolling(window=5, min_periods=1).mean()
            )
            
train['winner_id'] = train.apply(lambda row: 1 if row['winner_id'] == row['team1_id'] else 0, axis=1)
train['toss winner'] = train.apply(lambda row: 1 if row['toss winner'] == row['team1'] else 0, axis=1)
train['toss decision'] = train['toss decision'].apply(lambda x: 1 if x == 'bat' else 0)
train = train.drop(columns=['team1', 'team2', 'winner', 'series_name'])

In [11]:
def mean_ordinal_encode(train, target_col, columns_to_encode):
    encoding_mapsi = {}
    for col in columns_to_encode:
        means = train.groupby(col)[target_col].mean()
        train[f'{col}_encoded'] = train[col].map(means)
        encoding_mapsi[col] = means.to_dict()
    return train, encoding_mapsi

columns_to_encode = ['team1_id', 'team2_id', 'venue', 'city', 'lighting', 'season', 'ground_id']


train, encoding_mapsi = mean_ordinal_encode(train, 'winner_id', columns_to_encode)
train=train.drop(columns=[ 'venue', 'city', 'lighting', 'season', 'ground_id'])


In [12]:
missing_percentage = train.isnull().mean() * 100

# Print the percentage of missing values
print(missing_percentage)

match id                     0.0
team1_id                     0.0
team1_roster_ids             0.0
team2_id                     0.0
team2_roster_ids             0.0
winner_id                    0.0
toss winner                  0.0
toss decision                0.0
match_dt                     0.0
team_count_50runs_last15     0.0
team_winp_last5              0.0
team1only_avg_runs_last15    0.0
team1_winp_team2_last15      0.0
ground_avg_runs_last15       0.0
team1_id_encoded             0.0
team2_id_encoded             0.0
venue_encoded                0.0
city_encoded                 0.0
lighting_encoded             0.0
season_encoded               0.0
ground_id_encoded            0.0
dtype: float64


# test preprocessing

In [13]:
for column in test.columns:
        if test[column].isnull().any():  
            test[column] = test[column].fillna(
                test[column].rolling(window=5, min_periods=1).mean()
            )
            
#train['winner_id'] = train.apply(lambda row: 1 if row['winner_id'] == row['team1_id'] else 0, axis=1)
test['toss winner'] = test.apply(lambda row: 1 if row['toss winner'] == row['team1'] else 0, axis=1)
test['toss decision'] = train['toss decision'].apply(lambda x: 1 if x == 'bat' else 0)
test = test.drop(columns=['team1', 'team2', 'series_name'])

In [14]:
columns_to_encode = ['team1_id', 'team2_id', 'venue', 'city', 'lighting', 'season', 'ground_id']
def apply_mean_ordinal_encoding(test, encoding_maps, columns_to_encode):
    for col in columns_to_encode:
        if col in encoding_maps:
            #test[col] = test[col].map(encoding_maps[col])
            test[f'{col}_encoded'] = test[col].map(encoding_maps[col])
    return test

# Perform mean ordinal encoding on the match DataFrame
test = apply_mean_ordinal_encoding(test, encoding_mapsi, columns_to_encode)
test=test.drop(columns=[ 'venue', 'city', 'lighting', 'season', 'ground_id'])

In [15]:
for column in test.columns:
        if test[column].isnull().any():  
            test[column] = test[column].fillna(test[column].mode()[0])
            

In [16]:
missing_percentage = test.isnull().mean() * 100

# Print the percentage of missing values
print(missing_percentage)

match id                     0.0
team1_id                     0.0
team1_roster_ids             0.0
team2_id                     0.0
team2_roster_ids             0.0
toss winner                  0.0
toss decision                0.0
match_dt                     0.0
team_count_50runs_last15     0.0
team_winp_last5              0.0
team1only_avg_runs_last15    0.0
team1_winp_team2_last15      0.0
ground_avg_runs_last15       0.0
team1_id_encoded             0.0
team2_id_encoded             0.0
venue_encoded                0.0
city_encoded                 0.0
lighting_encoded             0.0
season_encoded               0.0
ground_id_encoded            0.0
dtype: float64


# Match preprocess

In [17]:
match['winner_id'] = match.apply(lambda row: 1 if row['winner_id'] == row['team1_id'] else 0, axis=1)
match['toss winner'] = match.apply(lambda row: 1 if row['toss winner'] == row['team1'] else 0, axis=1)
match['toss decision'] = match['toss decision'].apply(lambda x: 1 if x == 'bat' else 0)

def check_player_of_the_match(row):
    team1_roster_ids = set(str(row['team1_roster_ids']).split(':')) if pd.notnull(row['team1_roster_ids']) else set()
    team2_roster_ids = set(str(row['team2_roster_ids']).split(':')) if pd.notnull(row['team2_roster_ids']) else set()
    player_of_the_match_id = str(row['player_of_the_match_id'])

    if player_of_the_match_id in team1_roster_ids:
        return 1
    elif player_of_the_match_id in team2_roster_ids:
        return 0
    else:
        return None  # Handle cases where player_of_the_match_id is not found in either roster

# Apply the function to create 'player_of_the_match_team1' column
match['player_of_the_match_encoded'] = match.apply(check_player_of_the_match, axis=1)
mode_1 = match['player_of_the_match_encoded'].mode()[0]  
match['player_of_the_match_encoded'] = match['player_of_the_match_encoded'].fillna(mode_1)

mode_2 = match['city'].mode()[0]  
match['city'] = match['city'].fillna(mode_2)

mode_3 = match['umpire1'].mode()[0]  
match['umpire1'] = match['umpire1'].fillna(mode_3)

mode_4 = match['umpire2'].mode()[0]  
match['umpire2'] = match['umpire2'].fillna(mode_4)

match=match.drop(columns=['player_of_the_match_id'])

In [18]:
# match encoding from train encoding maps
columns_to_encode = ['team1_id', 'team2_id', 'venue', 'city', 'lighting', 'season', 'ground_id']

def mean_ordinal_encode(match, target_col, columns_to_encode):
    encoding_maps = {}
    for col in columns_to_encode:
        means = match.groupby(col)[target_col].mean()
        match[f'{col}_encoded'] = match[col].map(means)
        encoding_maps[col] = means.to_dict()
    return match, encoding_maps

match, encoding_maps = mean_ordinal_encode(match, 'winner_id', columns_to_encode)
for col in columns_to_encode:
    match[f'{col}_encoded'] = match[col].map(encoding_maps[col])
 
match=match.drop(columns=['team1_id', 'team2_id', 'venue', 'city', 'lighting', 'season', 'ground_id','team1', 'team2', 'winner', 'series_name'])
match.head()
    

,match id,by,win amount,toss winner,toss decision,match_dt,umpire1,umpire2,inning1_runs,inning1_wickets,...,series_type,winner_id,player_of_the_match_encoded,team1_id_encoded,team2_id_encoded,venue_encoded,city_encoded,lighting_encoded,season_encoded,ground_id_encoded
0,8638034,wickets,9.0,0,0,2021-01-01,Dn Mw,TJ Pe,148,6,...,other_domestic,0,1.0,0.400000,0.214286,0.000000,0.333333,0.485175,0.432692,0.333333
1,8588005,runs,7.0,0,0,2021-01-01,D Tr,PJ Ge,166,6,...,other_domestic,0,0.0,0.384615,0.642857,0.166667,0.166667,0.485175,0.432692,0.166667
2,8587837,wickets,4.0,1,1,2021-01-02,GJ Dn,PJ Ge,165,8,...,other_domestic,0,0.0,0.722222,0.529412,0.000000,0.600000,0.485175,0.432692,0.600000
3,8638041,wickets,2.0,0,0,2021-01-02,GA St,TJ Pe,153,7,...,other_domestic,0,1.0,0.400000,0.700000,0.000000,0.333333,0.485175,0.432692,0.333333
4,8587921,runs,21.0,0,0,2021-01-02,GA Ad,NR Je,164,5,...,other_domestic,1,1.0,0.533333,0.578947,1.000000,0.533333,0.553746,0.432692,0.533333


In [19]:
missing_percentage = match.isnull().mean() * 100

# Print the percentage of missing values
print(missing_percentage)

match id                       0.0
by                             0.0
win amount                     0.0
toss winner                    0.0
toss decision                  0.0
match_dt                       0.0
umpire1                        0.0
umpire2                        0.0
inning1_runs                   0.0
inning1_wickets                0.0
inning1_balls                  0.0
inning2_runs                   0.0
inning2_wickets                0.0
inning2_balls                  0.0
team1_roster_ids               0.0
team2_roster_ids               0.0
series_type                    0.0
winner_id                      0.0
player_of_the_match_encoded    0.0
team1_id_encoded               0.0
team2_id_encoded               0.0
venue_encoded                  0.0
city_encoded                   0.0
lighting_encoded               0.0
season_encoded                 0.0
ground_id_encoded              0.0
dtype: float64


# training regressor for predicting match features

In [20]:
columns_to_train=['umpire1','umpire2','by','win amount','inning1_runs','inning1_wickets','inning1_balls','inning2_runs','inning2_wickets','inning2_balls','series_type','player_of_the_match_encoded',]

columns_used_to_train=['team1_id_encoded','team2_id_encoded','toss winner','toss decision','venue_encoded','city_encoded','lighting_encoded','ground_id_encoded','season_encoded']

In [21]:
#mean ordinal encoding for categorical columns in columns to train
target_col='winner_id'
for col in columns_to_train:
    if match[col].dtype==object:
        means = match.groupby(col)[target_col].mean()
        match[f'{col}_encoded'] = match[col].map(means)
        
match.head()

,match id,by,win amount,toss winner,toss decision,match_dt,umpire1,umpire2,inning1_runs,inning1_wickets,...,team2_id_encoded,venue_encoded,city_encoded,lighting_encoded,season_encoded,ground_id_encoded,umpire1_encoded,umpire2_encoded,by_encoded,series_type_encoded
0,8638034,wickets,9.0,0,0,2021-01-01,Dn Mw,TJ Pe,148,6,...,0.214286,0.000000,0.333333,0.485175,0.432692,0.333333,0.333333,0.352941,0.002317,0.484509
1,8588005,runs,7.0,0,0,2021-01-01,D Tr,PJ Ge,166,6,...,0.642857,0.166667,0.166667,0.485175,0.432692,0.166667,0.428571,0.571429,0.991525,0.484509
2,8587837,wickets,4.0,1,1,2021-01-02,GJ Dn,PJ Ge,165,8,...,0.529412,0.000000,0.600000,0.485175,0.432692,0.600000,0.500000,0.571429,0.002317,0.484509
3,8638041,wickets,2.0,0,0,2021-01-02,GA St,TJ Pe,153,7,...,0.700000,0.000000,0.333333,0.485175,0.432692,0.333333,0.000000,0.352941,0.002317,0.484509
4,8587921,runs,21.0,0,0,2021-01-02,GA Ad,NR Je,164,5,...,0.578947,1.000000,0.533333,0.553746,0.432692,0.533333,0.619048,0.600000,0.991525,0.484509


In [22]:
match=match.drop(columns=['umpire1','umpire2','by','series_type'])

match.columns



Index(['match id', 'win amount', 'toss winner', 'toss decision', 'match_dt',
       'inning1_runs', 'inning1_wickets', 'inning1_balls', 'inning2_runs',
       'inning2_wickets', 'inning2_balls', 'team1_roster_ids',
       'team2_roster_ids', 'winner_id', 'player_of_the_match_encoded',
       'team1_id_encoded', 'team2_id_encoded', 'venue_encoded', 'city_encoded',
       'lighting_encoded', 'season_encoded', 'ground_id_encoded',
       'umpire1_encoded', 'umpire2_encoded', 'by_encoded',
       'series_type_encoded'],
      dtype='object')

In [23]:
columns_to_train=['umpire1_encoded','umpire2_encoded','by_encoded','win amount','inning1_runs','inning1_wickets','inning1_balls','inning2_runs','inning2_wickets','inning2_balls','series_type_encoded','player_of_the_match_encoded',]

columns_used_to_train=['team1_id_encoded','team2_id_encoded','toss winner','toss decision','venue_encoded','city_encoded','lighting_encoded','ground_id_encoded','season_encoded']

In [24]:
for col in columns_to_train:
    match[col]=match[col].astype(float)
    
for col in columns_used_to_train:
    match[col]=match[col].astype(float)
    train[col]=train[col].astype(float)
    test[col]=test[col].astype(float)

In [25]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

def train_xgb_models(match, target_column, columns_used_to_train):
    
    X = match[columns_used_to_train]
    y = match[target_column]
    print(f"currently working on the {target_column}")
    
    param_distributions = {
        'max_depth': randint(2, 10),
        'n_estimators': randint(100, 500),
        'learning_rate': uniform(0.01, 0.3),
        'subsample': uniform(0.5, 1),
        'colsample_bytree': uniform(0.5, 1)
    }
    
    
    
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
    
    
    
#     random_search = RandomizedSearchCV(xgb_model, param_distributions, n_iter=50, cv=5, n_jobs=-1, verbose=1, random_state=42)
#     random_search.fit(X, y)
    
    
#     best_model = random_search.best_estimator_
#     best_params = random_search.best_params_
#     print(f"Best parameters for target '{target_column}' is found")
    
    
    xgb_model.fit(X, y)
    
    return xgb_model


In [26]:
trained_models = {}
for target_col in columns_to_train:
    model = train_xgb_models(match, target_col, columns_used_to_train)
    print("model generated")
    X_train = train[columns_used_to_train]
    X_test = test[columns_used_to_train]
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)
    print("prediction generated")
    train[target_col] = predictions_train
    test[target_col] = predictions_test
    
    print(f"{target_col}prediction merged")
    trained_models[target_col] = model

currently working on the umpire1_encoded
model generated
prediction generated
umpire1_encodedprediction merged
currently working on the umpire2_encoded
model generated
prediction generated
umpire2_encodedprediction merged
currently working on the by_encoded
model generated
prediction generated
by_encodedprediction merged
currently working on the win amount
model generated
prediction generated
win amountprediction merged
currently working on the inning1_runs
model generated
prediction generated
inning1_runsprediction merged
currently working on the inning1_wickets
model generated
prediction generated
inning1_wicketsprediction merged
currently working on the inning1_balls
model generated
prediction generated
inning1_ballsprediction merged
currently working on the inning2_runs
model generated
prediction generated
inning2_runsprediction merged
currently working on the inning2_wickets
model generated
prediction generated
inning2_wicketsprediction merged
currently working on the inning2_ball

In [27]:
train.head()

,match id,team1_id,team1_roster_ids,team2_id,team2_roster_ids,winner_id,toss winner,toss decision,match_dt,team_count_50runs_last15,...,by_encoded,win amount,inning1_runs,inning1_wickets,inning1_balls,inning2_runs,inning2_wickets,inning2_balls,series_type_encoded,player_of_the_match_encoded
0,9331181,11283,9373356.0:7857520.0:4232164.0:4566540.0:329940...,12634,3500958.0:4231751.0:2735081.0:2035102.0:369833...,0,0.0,0.0,2022-10-20,1.666667,...,-0.083911,6.214568,110.538956,6.786717,119.756332,121.828529,1.790504,72.530617,0.480048,-0.038320
1,8797060,20,2089079.0:6139370.0:2076192.0:62432.0:2083409....,41,4690258.0:4069666.0:4230127.0:1942317.0:161392...,1,0.0,0.0,2022-01-23,1.285714,...,0.643580,52.040600,181.421738,5.912688,123.537392,157.428787,7.640804,113.432182,0.484047,0.682513
2,9433269,10576,3298427.0:2288789.0:7773338.0:3519011.0:368195...,8987,4003390.0:1749075.0:1626526.0:4172447.0:551672...,1,0.0,0.0,2023-06-02,0.857143,...,0.694852,22.822350,155.715988,6.227066,118.811699,142.110336,6.608956,109.944099,0.484010,0.854095
3,9587073,36084,8127230.0:4690328.0:4069666.0:7960847.0:469018...,36070,3462080.0:2436405.0:1798705.0:7550857.0:574247...,1,1.0,1.0,2023-09-10,2.166667,...,0.535525,15.574769,157.447311,7.812724,122.834694,143.299240,6.819320,112.868736,0.481062,0.619943
4,9516457,30407,8127181.0:197658.0:4239038.0:2398346.0:5053082...,48341,1958683.0:7491224.0:8059029.0:4377610.0:225245...,0,0.0,0.0,2023-04-13,0.818182,...,0.058142,10.025951,179.647461,7.402858,122.430740,168.281601,4.867413,116.394203,0.515514,0.362879


In [28]:
train.columns

Index(['match id', 'team1_id', 'team1_roster_ids', 'team2_id',
       'team2_roster_ids', 'winner_id', 'toss winner', 'toss decision',
       'match_dt', 'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'team1_id_encoded', 'team2_id_encoded',
       'venue_encoded', 'city_encoded', 'lighting_encoded', 'season_encoded',
       'ground_id_encoded', 'umpire1_encoded', 'umpire2_encoded', 'by_encoded',
       'win amount', 'inning1_runs', 'inning1_wickets', 'inning1_balls',
       'inning2_runs', 'inning2_wickets', 'inning2_balls',
       'series_type_encoded', 'player_of_the_match_encoded'],
      dtype='object')

In [29]:
test.columns

Index(['match id', 'team1_id', 'team1_roster_ids', 'team2_id',
       'team2_roster_ids', 'toss winner', 'toss decision', 'match_dt',
       'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'team1_id_encoded', 'team2_id_encoded',
       'venue_encoded', 'city_encoded', 'lighting_encoded', 'season_encoded',
       'ground_id_encoded', 'umpire1_encoded', 'umpire2_encoded', 'by_encoded',
       'win amount', 'inning1_runs', 'inning1_wickets', 'inning1_balls',
       'inning2_runs', 'inning2_wickets', 'inning2_balls',
       'series_type_encoded', 'player_of_the_match_encoded'],
      dtype='object')

# bowler and batter add

In [30]:
def roaster_processing(data):
    for i in range(len(data)):
        list1 = data['team1_roster_ids'].loc[i].split(':')
        list2 = data['team2_roster_ids'].loc[i].split(':')
    
        for j, value in enumerate(list1):
            col1 = f'team1_player_{j+1}'
            data.at[i,col1] = value
        
        for j, value in enumerate(list2):
            col2 = f'team2_player_{j+1}'
            data.at[i,col2] = value
    return data        


In [31]:
import numpy as np

def bowler_add(train, bowler):
    
    bowler['bowler_id'] = bowler['bowler_id'].astype(float)
    
    bowler_dict = {}

    for index, row in bowler.iterrows():
        bowler_id = row['bowler_id']
        if bowler_id not in bowler_dict:
            bowler_dict[bowler_id] = {'inning': [], 'runs': [], 'wicket_count': [], 'balls_bowled': [], 'economy': [], 'maiden': [], 'dots': [], 'Fours': [], 'Sixes': [], 'wides': [], 'noballs': []}

        bowler_dict[bowler_id]['inning'].append(row['inning'])
        bowler_dict[bowler_id]['runs'].append(row['runs'])
        bowler_dict[bowler_id]['wicket_count'].append(row['wicket_count'])
        bowler_dict[bowler_id]['balls_bowled'].append(row['balls_bowled'])
        bowler_dict[bowler_id]['economy'].append(row['economy'])
        bowler_dict[bowler_id]['maiden'].append(row['maiden'])
        bowler_dict[bowler_id]['dots'].append(row['dots'])
        bowler_dict[bowler_id]['Fours'].append(row['Fours'])
        bowler_dict[bowler_id]['Sixes'].append(row['Sixes'])
        bowler_dict[bowler_id]['wides'].append(row['wides'])
        bowler_dict[bowler_id]['noballs'].append(row['noballs'])

    for bowler_id, values in bowler_dict.items():
        bowler_dict[bowler_id]['average_inning'] = len(values['inning'])
        bowler_dict[bowler_id]['average_runs'] = sum(values['runs']) / len(values['runs'])
        bowler_dict[bowler_id]['average_wicket_count'] = sum(values['wicket_count']) / len(values['wicket_count'])
        bowler_dict[bowler_id]['average_balls_bowled'] = sum(values['balls_bowled']) / len(values['balls_bowled'])
        bowler_dict[bowler_id]['average_economy'] = sum(values['economy']) / len(values['economy'])
        bowler_dict[bowler_id]['average_maiden'] = sum(values['maiden']) / len(values['maiden'])
        bowler_dict[bowler_id]['average_dots'] = sum(values['dots']) / len(values['dots'])
        bowler_dict[bowler_id]['average_fours'] = sum(values['Fours']) / len(values['Fours'])
        bowler_dict[bowler_id]['average_sixes'] = sum(values['Sixes']) / len(values['Sixes'])
        bowler_dict[bowler_id]['average_wides'] = sum(values['wides']) / len(values['wides'])
        bowler_dict[bowler_id]['average_noballs'] = sum(values['noballs']) / len(values['noballs'])

    #print(bowler_dict)
    columns_to_include = [ 'bowler_inning_team1', 'bowler_runs_team1',
                         'wicket_count_team1', 'balls_bowled_team1', 'economy_team1', 'maiden_team1', 'bowler_dots_team1', 'bowler_Fours_team1',
                         'bowler_Sixes_team1', 'wides_team1', 'noballs_team1','bowler_inning_team2', 'bowler_runs_team2',
                         'wicket_count_team2', 'balls_bowled_team2', 'economy_team2', 'maiden_team2', 'bowler_dots_team2', 'bowler_Fours_team2',
                         'bowler_Sixes_team2', 'wides_team2', 'noballs_team2']
    for col in columns_to_include:
        train[col] = [None] * len(train)
    
    for index, row in train.iterrows():
        bowler_inning_team1 = []
        bowler_runs_team1 = []
        wicket_count_team1 = []
        balls_bowled_team1 = []
        economy_team1 = []
        maiden_team1 = []
        dots_team1 = []
        fours_team1 = []
        sixes_team1 = []
        wides_team1 = []
        noballs_team1 = []
        
        bowler_inning_team2 = []
        bowler_runs_team2 = []
        wicket_count_team2 = []
        balls_bowled_team2 = []
        economy_team2 = []
        maiden_team2 = []
        dots_team2 = []
        fours_team2 = []
        sixes_team2 = []
        wides_team2 = []
        noballs_team2 = []

        for i in range(1, 13):
            
            player1_id = row[f'team1_player_{i}']
            player2_id = row[f'team2_player_{i}']
            
            if player1_id is not None:
                player1_id = float(player1_id)
            if player2_id is not None:
                player2_id = float(player2_id)
        
            if player1_id in bowler_dict:
                #print("mil gaya bencho")
                bowler_inning_team1.append(bowler_dict[player1_id]['average_inning'])
                bowler_runs_team1.append(bowler_dict[player1_id]['average_runs'])
                wicket_count_team1.append(bowler_dict[player1_id]['average_wicket_count'])
                balls_bowled_team1.append(bowler_dict[player1_id]['average_balls_bowled'])
                economy_team1.append(bowler_dict[player1_id]['average_economy'])
                maiden_team1.append(bowler_dict[player1_id]['average_maiden'])
                dots_team1.append(bowler_dict[player1_id]['average_dots'])
                fours_team1.append(bowler_dict[player1_id]['average_fours'])
                sixes_team1.append(bowler_dict[player1_id]['average_sixes'])
                wides_team1.append(bowler_dict[player1_id]['average_wides'])
                noballs_team1.append(bowler_dict[player1_id]['average_noballs'])

            if player2_id in bowler_dict:
                #print("mil gaya bencho")
                bowler_inning_team2.append(bowler_dict[player2_id]['average_inning'])
                bowler_runs_team2.append(bowler_dict[player2_id]['average_runs'])
                wicket_count_team2.append(bowler_dict[player2_id]['average_wicket_count'])
                balls_bowled_team2.append(bowler_dict[player2_id]['average_balls_bowled'])
                economy_team2.append(bowler_dict[player2_id]['average_economy'])
                maiden_team2.append(bowler_dict[player2_id]['average_maiden'])
                dots_team2.append(bowler_dict[player2_id]['average_dots'])
                fours_team2.append(bowler_dict[player2_id]['average_fours'])
                sixes_team2.append(bowler_dict[player2_id]['average_sixes'])
                wides_team2.append(bowler_dict[player2_id]['average_wides'])
                noballs_team2.append(bowler_dict[player2_id]['average_noballs'])

        train.at[index, 'bowler_inning_team1'] = np.sum(bowler_inning_team1) if bowler_inning_team1 else None
        train.at[index, 'bowler_runs_team1'] = np.sum(bowler_runs_team1) if bowler_runs_team1 else None
        train.at[index, 'wicket_count_team1'] = np.sum(wicket_count_team1) if wicket_count_team1 else None
        train.at[index, 'balls_bowled_team1'] = np.sum(balls_bowled_team1) if balls_bowled_team1 else None
        train.at[index, 'economy_team1'] = np.sum(economy_team1) if economy_team1 else None
        train.at[index, 'maiden_team1'] = np.sum(maiden_team1) if maiden_team1 else None
        train.at[index, 'bowler_dots_team1'] = np.sum(dots_team1) if dots_team1 else None
        train.at[index, 'bowler_Fours_team1'] = np.sum(fours_team1) if fours_team1 else None
        train.at[index, 'bowler_Sixes_team1'] = np.sum(sixes_team1) if sixes_team1 else None
        train.at[index, 'wides_team1'] = np.sum(wides_team1) if wides_team1 else None
        train.at[index, 'noballs_team1'] = np.sum(noballs_team1) if noballs_team1 else None
        
        train.at[index, 'bowler_inning_team2'] = np.sum(bowler_inning_team2) if bowler_inning_team2 else None
        train.at[index, 'bowler_runs_team2'] = np.sum(bowler_runs_team2) if bowler_runs_team2 else None
        train.at[index, 'wicket_count_team2'] = np.sum(wicket_count_team2) if wicket_count_team2 else None
        train.at[index, 'balls_bowled_team2'] = np.sum(balls_bowled_team2) if balls_bowled_team2 else None
        train.at[index, 'economy_team2'] = np.sum(economy_team2) if economy_team2 else None
        train.at[index, 'maiden_team2'] = np.sum(maiden_team2) if maiden_team2 else None
        train.at[index, 'bowler_dots_team2'] = np.sum(dots_team2) if dots_team2 else None
        train.at[index, 'bowler_Fours_team2'] = np.sum(fours_team2) if fours_team2 else None
        train.at[index, 'bowler_Sixes_team2'] = np.sum(sixes_team2) if sixes_team2 else None
        train.at[index, 'wides_team2'] = np.sum(wides_team2) if wides_team2 else None
        train.at[index, 'noballs_team2'] = np.sum(noballs_team2) if noballs_team2 else None
    return train

In [32]:
def batter_add(train, batter):
    batter = batter.fillna(0)
    batter['batsman_id'] = batter['batsman_id'].astype(float)
    
    batter_dict = {}

    for index, row in batter.iterrows():
        batsman_id = row['batsman_id']
        if batsman_id not in batter_dict:
            batter_dict[batsman_id] = {'inning': [], 'runs': [], 'balls_faced': [], 'strike_rate': [], 'Fours': [], 'Sixes': []}

        batter_dict[batsman_id]['inning'].append(row['inning'])
        batter_dict[batsman_id]['runs'].append(row['runs'])
        batter_dict[batsman_id]['balls_faced'].append(row['balls_faced'])
        batter_dict[batsman_id]['strike_rate'].append(row['strike_rate'])
        batter_dict[batsman_id]['Fours'].append(row['Fours'])
        batter_dict[batsman_id]['Sixes'].append(row['Sixes'])

    for batsman_id, values in batter_dict.items():
        batter_dict[batsman_id]['average_inning'] = len(values['inning'])
        batter_dict[batsman_id]['average_runs'] = sum(values['runs']) / len(values['runs'])
        batter_dict[batsman_id]['average_balls_faced'] = sum(values['balls_faced']) / len(values['balls_faced'])
        batter_dict[batsman_id]['average_strike_rate'] = sum(values['strike_rate']) / len(values['strike_rate'])
        batter_dict[batsman_id]['average_fours'] = sum(values['Fours']) / len(values['Fours'])
        batter_dict[batsman_id]['average_sixes'] = sum(values['Sixes']) / len(values['Sixes'])

    #print(batter_dict)
    columns_to_include = ['batter_inning_team1', 'batter_runs_team1', 'batter_balls_faced_team1', 'strike_rate_team1', 'batter_fours_team1', 'batter_sixes_team1',
                          'batter_inning_team2', 'batter_runs_team2', 'batter_balls_faced_team2', 'strike_rate_team2', 'batter_fours_team2', 'batter_sixes_team2']
    
    for col in columns_to_include:
        train[col] = [None] * len(train)
    
    for index, row in train.iterrows():
        batter_inning_team1 = []
        batter_runs_team1 = []
        batter_balls_faced_team1 = []
        strike_rate_team1 = []
        batter_fours_team1 = []
        batter_sixes_team1 = []
        
        batter_inning_team2 = []
        batter_runs_team2 = []
        batter_balls_faced_team2 = []
        strike_rate_team2 = []
        batter_fours_team2 = []
        batter_sixes_team2 = []

        for i in range(1, 13):
            player1_id = row[f'team1_player_{i}']
            player2_id = row[f'team2_player_{i}']
            
            if player1_id is not None:
                player1_id = float(player1_id)
            if player2_id is not None:
                player2_id = float(player2_id)
        
            if player1_id in batter_dict:
                batter_inning_team1.append(batter_dict[player1_id]['average_inning'])
                batter_runs_team1.append(batter_dict[player1_id]['average_runs'])
                batter_balls_faced_team1.append(batter_dict[player1_id]['average_balls_faced'])
                strike_rate_team1.append(batter_dict[player1_id]['average_strike_rate'])
                batter_fours_team1.append(batter_dict[player1_id]['average_fours'])
                batter_sixes_team1.append(batter_dict[player1_id]['average_sixes'])

            if player2_id in batter_dict:
                batter_inning_team2.append(batter_dict[player2_id]['average_inning'])
                batter_runs_team2.append(batter_dict[player2_id]['average_runs'])
                batter_balls_faced_team2.append(batter_dict[player2_id]['average_balls_faced'])
                strike_rate_team2.append(batter_dict[player2_id]['average_strike_rate'])
                batter_fours_team2.append(batter_dict[player2_id]['average_fours'])
                batter_sixes_team2.append(batter_dict[player2_id]['average_sixes'])

        train.at[index, 'batter_inning_team1'] = np.sum(batter_inning_team1) if batter_inning_team1 else None
        train.at[index, 'batter_runs_team1'] = np.sum(batter_runs_team1) if batter_runs_team1 else None
        train.at[index, 'batter_balls_faced_team1'] = np.sum(batter_balls_faced_team1) if batter_balls_faced_team1 else None
        train.at[index, 'strike_rate_team1'] = np.sum(strike_rate_team1) if strike_rate_team1 else None
        train.at[index, 'batter_fours_team1'] = np.sum(batter_fours_team1) if batter_fours_team1 else None
        train.at[index, 'batter_sixes_team1'] = np.sum(batter_sixes_team1) if batter_sixes_team1 else None
        
        train.at[index, 'batter_inning_team2'] = np.sum(batter_inning_team2) if batter_inning_team2 else None
        train.at[index, 'batter_runs_team2'] = np.sum(batter_runs_team2) if batter_runs_team2 else None
        train.at[index, 'batter_balls_faced_team2'] = np.sum(batter_balls_faced_team2) if batter_balls_faced_team2 else None
        train.at[index, 'strike_rate_team2'] = np.sum(strike_rate_team2) if strike_rate_team2 else None
        train.at[index, 'batter_fours_team2'] = np.sum(batter_fours_team2) if batter_fours_team2 else None
        train.at[index, 'batter_sixes_team2'] = np.sum(batter_sixes_team2) if batter_sixes_team2 else None
        
    return train

In [33]:

train=roaster_processing(train)
train=bowler_add(train, bowler)
train=batter_add(train, batter)


In [34]:
train.columns

Index(['match id', 'team1_id', 'team1_roster_ids', 'team2_id',
       'team2_roster_ids', 'winner_id', 'toss winner', 'toss decision',
       'match_dt', 'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'team1_id_encoded', 'team2_id_encoded',
       'venue_encoded', 'city_encoded', 'lighting_encoded', 'season_encoded',
       'ground_id_encoded', 'umpire1_encoded', 'umpire2_encoded', 'by_encoded',
       'win amount', 'inning1_runs', 'inning1_wickets', 'inning1_balls',
       'inning2_runs', 'inning2_wickets', 'inning2_balls',
       'series_type_encoded', 'player_of_the_match_encoded', 'team1_player_1',
       'team1_player_2', 'team1_player_3', 'team1_player_4', 'team1_player_5',
       'team1_player_6', 'team1_player_7', 'team1_player_8', 'team1_player_9',
       'team1_player_10', 'team1_player_11', 'team1_player_12',
       'team2_player_1', 'team2_player_2', 'team2_player_3', 'team2_play

In [35]:
test=roaster_processing(test)
test=bowler_add(test, bowler)
test=batter_add(test, batter)
test.columns

Index(['match id', 'team1_id', 'team1_roster_ids', 'team2_id',
       'team2_roster_ids', 'toss winner', 'toss decision', 'match_dt',
       'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'team1_id_encoded', 'team2_id_encoded',
       'venue_encoded', 'city_encoded', 'lighting_encoded', 'season_encoded',
       'ground_id_encoded', 'umpire1_encoded', 'umpire2_encoded', 'by_encoded',
       'win amount', 'inning1_runs', 'inning1_wickets', 'inning1_balls',
       'inning2_runs', 'inning2_wickets', 'inning2_balls',
       'series_type_encoded', 'player_of_the_match_encoded', 'team1_player_1',
       'team1_player_2', 'team1_player_3', 'team1_player_4', 'team1_player_5',
       'team1_player_6', 'team1_player_7', 'team1_player_8', 'team1_player_9',
       'team1_player_10', 'team1_player_11', 'team2_player_1',
       'team2_player_2', 'team2_player_3', 'team2_player_4', 'team2_player_5',
       

In [36]:
columns_to_delete=['team1_player_1', 'team1_player_2',
       'team1_player_3', 'team1_player_4', 'team1_player_5', 'team1_player_6',
       'team1_player_7', 'team1_player_8', 'team1_player_9', 'team1_player_10',
       'team1_player_11', 'team1_player_12', 'team2_player_1',
       'team2_player_2', 'team2_player_3', 'team2_player_4', 'team2_player_5',
       'team2_player_6', 'team2_player_7', 'team2_player_8', 'team2_player_9',
       'team2_player_10', 'team2_player_11', 'team2_player_12', 'team1_roster_ids', 'team2_roster_ids',]

In [37]:
train=train.drop(columns=columns_to_delete)
train.columns

Index(['match id', 'team1_id', 'team2_id', 'winner_id', 'toss winner',
       'toss decision', 'match_dt', 'team_count_50runs_last15',
       'team_winp_last5', 'team1only_avg_runs_last15',
       'team1_winp_team2_last15', 'ground_avg_runs_last15', 'team1_id_encoded',
       'team2_id_encoded', 'venue_encoded', 'city_encoded', 'lighting_encoded',
       'season_encoded', 'ground_id_encoded', 'umpire1_encoded',
       'umpire2_encoded', 'by_encoded', 'win amount', 'inning1_runs',
       'inning1_wickets', 'inning1_balls', 'inning2_runs', 'inning2_wickets',
       'inning2_balls', 'series_type_encoded', 'player_of_the_match_encoded',
       'bowler_inning_team1', 'bowler_runs_team1', 'wicket_count_team1',
       'balls_bowled_team1', 'economy_team1', 'maiden_team1',
       'bowler_dots_team1', 'bowler_Fours_team1', 'bowler_Sixes_team1',
       'wides_team1', 'noballs_team1', 'bowler_inning_team2',
       'bowler_runs_team2', 'wicket_count_team2', 'balls_bowled_team2',
       'economy_te

In [38]:
test=test.drop(columns=columns_to_delete)
#test=test.drop(columns=[' team2_player_13'])
test.columns

Index(['match id', 'team1_id', 'team2_id', 'toss winner', 'toss decision',
       'match_dt', 'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'team1_id_encoded', 'team2_id_encoded',
       'venue_encoded', 'city_encoded', 'lighting_encoded', 'season_encoded',
       'ground_id_encoded', 'umpire1_encoded', 'umpire2_encoded', 'by_encoded',
       'win amount', 'inning1_runs', 'inning1_wickets', 'inning1_balls',
       'inning2_runs', 'inning2_wickets', 'inning2_balls',
       'series_type_encoded', 'player_of_the_match_encoded', 'team2_player_13',
       'bowler_inning_team1', 'bowler_runs_team1', 'wicket_count_team1',
       'balls_bowled_team1', 'economy_team1', 'maiden_team1',
       'bowler_dots_team1', 'bowler_Fours_team1', 'bowler_Sixes_team1',
       'wides_team1', 'noballs_team1', 'bowler_inning_team2',
       'bowler_runs_team2', 'wicket_count_team2', 'balls_bowled_team2',
       'econ

In [39]:
test=test.drop(columns=['team2_player_13'])


In [40]:
train=train.drop(columns=['match_dt'])
test=test.drop(columns=['match_dt'])

In [41]:
train = train.astype(float)
test=test.astype(float)

# Fill null values with the mean of each column
train = train.fillna(train.mean())
test=test.fillna(test.mean())

In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 64 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   match id                     948 non-null    float64
 1   team1_id                     948 non-null    float64
 2   team2_id                     948 non-null    float64
 3   winner_id                    948 non-null    float64
 4   toss winner                  948 non-null    float64
 5   toss decision                948 non-null    float64
 6   team_count_50runs_last15     948 non-null    float64
 7   team_winp_last5              948 non-null    float64
 8   team1only_avg_runs_last15    948 non-null    float64
 9   team1_winp_team2_last15      948 non-null    float64
 10  ground_avg_runs_last15       948 non-null    float64
 11  team1_id_encoded             948 non-null    float64
 12  team2_id_encoded             948 non-null    float64
 13  venue_encoded       

In [43]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271 entries, 0 to 270
Data columns (total 63 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   match id                     271 non-null    float64
 1   team1_id                     271 non-null    float64
 2   team2_id                     271 non-null    float64
 3   toss winner                  271 non-null    float64
 4   toss decision                271 non-null    float64
 5   team_count_50runs_last15     271 non-null    float64
 6   team_winp_last5              271 non-null    float64
 7   team1only_avg_runs_last15    271 non-null    float64
 8   team1_winp_team2_last15      271 non-null    float64
 9   ground_avg_runs_last15       271 non-null    float64
 10  team1_id_encoded             271 non-null    float64
 11  team2_id_encoded             271 non-null    float64
 12  venue_encoded                271 non-null    float64
 13  city_encoded        

In [48]:
train.to_csv("train_final_3.csv",index=False)
test.to_csv("test_final_3.csv",index=False)

In [45]:
# import pandas as pd
# train=pd.read_csv('/kaggle/input/kuch-bhi/train_final.csv')

In [46]:
# y_train=train['winner_id']
# X_train=train.drop(columns=['winner_id'])

In [47]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.ensemble import StackingClassifier
# from sklearn.metrics import classification_report

In [49]:
test.head()

,match id,team1_id,team2_id,toss winner,toss decision,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,...,batter_balls_faced_team1,strike_rate_team1,batter_fours_team1,batter_sixes_team1,batter_inning_team2,batter_runs_team2,batter_balls_faced_team2,strike_rate_team2,batter_fours_team2,batter_sixes_team2
0,9250275.0,28594.0,90.0,0.0,0.0,0.000000,0.019608,171.066667,0.00,127.500,...,65.000000,555.082000,13.800000,2.800000,41.0,170.783333,134.383333,1193.269000,13.033333,6.116667
1,9262189.0,36084.0,36098.0,0.0,0.0,0.615385,0.344262,151.285714,66.67,153.500,...,133.307066,1354.593792,13.531288,9.915395,430.0,163.589714,131.608560,1204.013564,11.936341,8.420124
2,9128776.0,30428.0,48334.0,1.0,0.0,0.842105,0.753086,171.066667,100.00,179.625,...,139.069437,1147.400308,16.267339,7.949675,428.0,167.695640,130.845687,1191.326554,12.871510,7.728367
3,9586919.0,36098.0,36112.0,0.0,0.0,0.285714,1.487805,166.000000,75.00,155.500,...,128.639123,974.000408,10.689029,5.759284,282.0,159.766639,121.934233,1204.601805,12.286929,7.238745
4,9128538.0,30414.0,48341.0,0.0,0.0,2.375000,0.310330,169.933333,0.00,164.125,...,144.259667,1272.238543,15.008921,10.002741,397.0,177.176979,136.529055,1072.063700,14.404248,7.783830
